In [1]:
from youtube_transcript_api import YouTubeTranscriptApi
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.indexes import  VectorstoreIndexCreator
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.llms import OpenAI
from langchain.memory import VectorStoreRetrieverMemory
from langchain.llms import GPT4All
from langchain import ConversationChain, PromptTemplate
import os

In [2]:
video_links = ["9lVj_DZm36c", "ZUN3AFNiEgc", "8KtDLu4a-EM"]

if os.path.exists('transcripts'):
    print('Directory already exists')
else:
    os.mkdir('transcripts')
for video_id in video_links:
    dir = os.path.join('transcripts', video_id)
    transcript = YouTubeTranscriptApi.get_transcript(video_id)

    with open(dir+'.txt', 'w') as f:
     for line in transcript:
            f.write(f"{line['text']}\n")



Directory already exists


In [3]:
loader = DirectoryLoader(path='./', glob = "**/*.txt", loader_cls=TextLoader,
                         show_progress=True)

In [4]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [5]:
index = VectorstoreIndexCreator(embedding=embeddings).from_loaders([loader])
retriever = index.vectorstore.as_retriever(search_kwargs=dict(k=5))
memory = VectorStoreRetrieverMemory(retriever=retriever)


100%|██████████| 3/3 [00:00<00:00, 103.46it/s]


In [19]:

llm = GPT4All(model="./ggml-mpt-7b-instruct.bin", top_p=0.15, top_k=0,  temp=0.5, repeat_penalty=1.1)


Found model file.


In [20]:
_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.
Do not make up answers and provide only information that you have.
Relevant pieces of previous conversation:
{history}

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: {input}
AI:"""

PROMPT = PromptTemplate(
    input_variables=[ "history", "input"], template=_DEFAULT_TEMPLATE
)


In [21]:
import torch


conversation_with_summary = ConversationChain(
    llm=llm, 
    prompt=PROMPT,
    # We set a very low max_token_limit for the purposes of testing.
    memory = memory
    )


with torch.inference_mode():

    print(conversation_with_summary.predict(input = "How large is california"))

In [10]:
INSTRUCTION_KEY = "### Instruction:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"
INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
PROMPT_FOR_GENERATION_FORMAT = """{intro}
{instruction_key}
{instruction}
{response_key}
""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
)

In [15]:
import torch


with torch.inference_mode():
    print(llm(PROMPT_FOR_GENERATION_FORMAT.format(instruction = "what are the specifications of lenovo slim 7i 2023 edition")))


1) Processor : Intel Core i5 10th Gen processor (1035G1) with 4 Cores and 8 Threads, Turbo Boost Upto 3.8 GHz, 6 MB Cache Memory 2) Operating System - Windows 11 Home Edition 64 Bit OS 3) Display Size- 15 inch FHD Anti Glare IPS display with 1920 x 1080 pixels resolution 4) Graphics Card : Intel Integrated GPU 5) RAM & Storage: 8 GB DDR4 2400 MHz SDRAM, 1 TB HDD 7200 RPM, 2 Slots for M.2 SSD 6) Battery - Li ion battery
1) Processor : Intel Core i5 10th Gen processor (1035G1) with 4 Cores and 8 Threads, Turbo Boost Upto 3.8 GHz, 6 MB Cache Memory 2) Operating System - Windows 11 Home Edition 64 Bit OS 3) Display Size- 15 inch FHD Anti Glare IPS display with 1920 x 1080 pixels resolution 4) Graphics Card : Intel Integrated GPU 5) RAM & Storage: 8 GB DDR4 2400 MHz SDRAM, 1 TB HDD 7200 RPM, 2 Slots for M.2 SSD 6) Battery - Li ion battery


In [18]:
repo_name = "rustformers/mpt-7b-ggml"
file_name = "mpt-7b-instruct-q5_1-ggjt.bin"

from llm_rs import AutoModel,SessionConfig,GenerationConfig,Precision

session_config = SessionConfig(threads=2,batch_size=2)
model = AutoModel.from_pretrained(repo_name, model_file=file_name, session_config=session_config,verbose=True, )


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:


def process_stream(instruction, temperature, top_p, top_k, max_new_tokens, seed):

    prompt=f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{instruction}
### Response:
Answer:"""
    generation_config = GenerationConfig(seed=seed,temperature=temperature,top_p=top_p,top_k=top_k,max_new_tokens=max_new_tokens)
    response = ""
    streamer = model.stream(prompt=prompt,generation_config=generation_config)
    for new_text in streamer:
        response += new_text
        yield response

stream = process_stream(instruction= "How big is india", temperature= 0.5, top_p=0.15, top_k=0, max_new_tokens=512, seed = 42)

for i in stream:
    print(i)

  

PanicException: called `Option::unwrap()` on a `None` value